In [84]:
import os
import pickle
from collections import Counter
from tqdm.auto import 

In [7]:
axioms = ["TFC2"]
data_home = "/ssd2/arthur/bert-axioms/"
cut = "cut"
docs_path = os.path.join(data_home, "docs/msmarco-docs.tokenized.cut.tsv")
offset_path = docs_path + ".offset"
offset_dict = pickle.load(open(offset_path, 'rb'))

In [90]:
top_100_path = os.path.join(data_home, "runs/QL_test-{}.run".format(cut))
all_docs_p = os.path.join(data_home, "tmp", "all_docs.pkl")
docs_lens_p = os.path.join(data_home, "tmp", "docs_lens.pkl")
tuples_p = os.path.join(data_home, "tmp", "tuples.pkl")
all_docs = pickle.load(open(all_docs_p, 'rb'))
docs_lens = pickle.load(open(docs_lens_p, 'rb'))
tuples = pickle.load(open(tuples_p, 'rb'))
scores = {}
for line in open(top_100_path):
    topic_id, _, doc_id, _, score, _ = line.split(" ")
    scores["{}-{}".format(topic_id, doc_id)] = float(score)


In [75]:
queries_file = os.path.join(data_home, "queries/test.tokenized.tsv")
diagnostics_path = os.path.join(data_home, "diagnostics")
all_lines = [(x.split("\t")[0], x.split("\t")[1].strip().split(" ")) for x in open(queries_file).readlines()]
n_cpus = 48
# add padding if needed
if len(all_lines) % n_cpus != 0:
    print("padding topics with %i empty topics" % (len(all_lines) % n_cpus))
while len(all_lines) % n_cpus != 0:
    all_lines.append((None, None))
chunk_size = len(all_lines)//n_cpus
print("Each process will read %i topics" % chunk_size)
chunks = [all_lines[i*chunk_size: (i+1)*chunk_size] for i in range(n_cpus)]

padding topics with 22 empty topics
Each process will read 33 topics


In [94]:
len(tuples['201376'])

100

In [101]:
[x[0] for x in chunk]

['201376',
 '54544',
 '178627',
 '257309',
 '1101276',
 '342285',
 '372586',
 '206117',
 '141472',
 '352818',
 '14151',
 '323998',
 '289812',
 '333486',
 '73257',
 '1090170',
 '127876',
 '259417',
 '1101271',
 '335710',
 '1090086',
 '1090077',
 '1090072',
 '1090054',
 '1089964',
 '1101236',
 '1089810',
 '1089776',
 '1089763',
 '1089691',
 '1089688',
 '1089674',
 '1089576']

In [99]:
chunk[:1]

[('201376', ['here', 'there', 'be', 'dragons', 'comic'])]

In [91]:
args  = {"delta": 10}
def TFC2(chunk, all_docs, tuples, docs_lens, args, qrels):
    instances = []
    qrels = scores
    for sample in tqdm(chunk[:2]):
        topic_id, query_terms = sample
        query_terms = set(query_terms)
        fullfils = 0
        for di_id in tuples[topic_id]:
            di_text = [w for w in all_docs[di_id] if w in query_terms]
            if len(di_text) == 0:
                continue
            di_terms_counter = Counter(di_text)
            sum_di_terms = sum(di_terms_counter.values())
            for dj_id in tuples[topic_id]:
                if dj_id == di_id:
                    continue
                if abs(docs_lens[di_id] - docs_lens[dj_id]) > args["delta"]:
                    continue
                dj_text = [w for w in all_docs[dj_id] if w in query_terms]
                if len(dj_text) == 0:
                    continue
                dj_terms_counter = Counter(dj_text)
                sum_dj_terms = sum(dj_terms_counter.values())
                for dk_id in tuples[topic_id]:
                    if dk_id == di_id or dk_id == dj_id:
                        continue
                    if (abs(docs_lens[dk_id] - docs_lens[dj_id]) > args["delta"]
                       or abs(docs_lens[dk_id] - docs_lens[di_id]) > args["delta"]):  # noqa: W503
                        continue
                    dk_text = [w for w in all_docs[dk_id] if w in query_terms]
                    if len(dk_text) == 0:
                        continue
                    dk_terms_counter = Counter(dk_text)
                    sum_dk_terms = sum(dk_terms_counter.values())
                    if not (sum_dk_terms > sum_dj_terms and sum_dj_terms > sum_di_terms and sum_di_terms > 0):
                        continue
                    flag = False
                    for w in query_terms:
                        if (dj_terms_counter[w] - di_terms_counter[w]) != (dk_terms_counter[w] - dj_terms_counter[w]):
                            flag = True
                            break
                    if flag:
                        continue
                    instances.append((topic_id, di_id, dj_id, dk_id))
                    di_score = qrels["{}-{}".format(topic_id, di_id)]
                    dj_score = qrels["{}-{}".format(topic_id, dj_id)]
                    dk_score = qrels["{}-{}".format(topic_id, dk_id)]
                    if dj_score - di_score > dk_score - dj_score:
                        fullfils += 1
    return instances

('201376', ['here', 'there', 'be', 'dragons', 'comic'])

In [122]:
all_docs_per_chunk = []
for chunk in tqdm(chunks):
    chunk_dict = {}
    for topic, _ in chunk:
        if topic is None: 
            continue
        for doc in tuples[topic]:
            chunk_dict[doc] = all_docs[doc]
    all_docs_per_chunk.append(chunk_dict)
    

In [124]:
len(all_docs_per_chunk) == len(chunks)


True